# 训练Xception模型，并保存权重

1 使用 Xception

2 冻结训练分类器

3 finetune训练整个网络

### 1 import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Using TensorFlow backend.


### 2 build pretrained model

In [2]:
input_tensor = Input((299, 299, 3))
x = Lambda(inception_v3.preprocess_input)(input_tensor)

base_model = InceptionV3(input_tensor=x, weights='imagenet', include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(2, activation='softmax')(p_out)

for layer in base_model.layers:  
     layer.trainable = False

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

### 3 build param

In [3]:
batch_size = 32
epochs = 5

total_imgs_num = 5232
train_imgs_num = 5232 - 524
valid_imgs_num = 524

img_size = (299, 299)

### 4 build data & train

In [4]:
img_gen_t = ImageDataGenerator(#rotation_range=180,
                               #width_shift_range=0.3,
                               #height_shift_range=0.3,
                               #zoom_range=0.3,
                               #horizontal_flip=True,
                               #vertical_flip=True)
)
train_generator = img_gen_t.flow_from_directory("../dataset/chest_xray/train-ready", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../dataset/chest_xray/valid-ready", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("InceptionV3_first_train.h5")

Found 4708 images belonging to 2 classes.
Found 524 images belonging to 2 classes.
Epoch 1/5
147/147 [==============================] - 76s 517ms/step - loss: 0.3616 - acc: 0.8437 - val_loss: 0.4233 - val_acc: 0.8262
Epoch 2/5
147/147 [==============================] - 72s 487ms/step - loss: 0.2265 - acc: 0.9147 - val_loss: 0.4333 - val_acc: 0.7988
Epoch 3/5
147/147 [==============================] - 72s 488ms/step - loss: 0.2062 - acc: 0.9245 - val_loss: 0.6367 - val_acc: 0.6660
Epoch 4/5
147/147 [==============================] - 72s 488ms/step - loss: 0.1905 - acc: 0.9260 - val_loss: 0.4823 - val_acc: 0.7812
Epoch 5/5
147/147 [==============================] - 72s 489ms/step - loss: 0.1713 - acc: 0.9326 - val_loss: 0.5644 - val_acc: 0.7715


### 5 Finetune

In [5]:
input_tensor = Input((299, 299, 3))
x = Lambda(inception_v3.preprocess_input)(input_tensor)

base_model = InceptionV3(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(2, activation='softmax')(p_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('InceptionV3_first_train.h5')
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         lambda_2[0][0]                   
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation

In [6]:
batch_size = 4
epochs = 10

total_imgs_num = 5232
train_imgs_num = 5232 - 524
valid_imgs_num = 524

img_size = (299, 299)

img_gen_t = ImageDataGenerator(#rotation_range=180,
                               #width_shift_range=0.3,
                               #height_shift_range=0.3,
                               #zoom_range=0.3,
                               #horizontal_flip=True,
                               #vertical_flip=True
)
train_generator = img_gen_t.flow_from_directory("../dataset/chest_xray/train-ready", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../dataset/chest_xray/valid-ready", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("InceptionV3_finetune.h5")

Found 4708 images belonging to 2 classes.
Found 524 images belonging to 2 classes.
Epoch 1/10
1177/1177 [==============================] - 346s 294ms/step - loss: 0.2894 - acc: 0.8857 - val_loss: 0.0708 - val_acc: 0.9733
Epoch 2/10
1177/1177 [==============================] - 334s 284ms/step - loss: 0.1460 - acc: 0.9535 - val_loss: 0.0825 - val_acc: 0.9695
Epoch 3/10
1177/1177 [==============================] - 334s 284ms/step - loss: 0.0924 - acc: 0.9692 - val_loss: 0.0361 - val_acc: 0.9885
Epoch 4/10
1177/1177 [==============================] - 335s 284ms/step - loss: 0.0775 - acc: 0.9779 - val_loss: 0.0357 - val_acc: 0.9847
Epoch 5/10
1177/1177 [==============================] - 334s 284ms/step - loss: 0.0653 - acc: 0.9771 - val_loss: 0.0207 - val_acc: 0.9885
Epoch 6/10
1177/1177 [==============================] - 335s 284ms/step - loss: 0.0614 - acc: 0.9815 - val_loss: 0.0206 - val_acc: 0.9943
Epoch 7/10
1177/1177 [==============================] - 335s 284ms/step - loss: 0.0463 - 

KeyboardInterrupt: 

In [7]:
batch_size = 4
epochs = 1

total_imgs_num = 4708 + 624
train_imgs_num = 4708
valid_imgs_num = 624

img_size = (299, 299)

img_gen_t = ImageDataGenerator(#rotation_range=180,
                               #width_shift_range=0.3,
                               #height_shift_range=0.3,
                               #zoom_range=0.3,
                               #horizontal_flip=True,
                               #vertical_flip=True)
)
train_generator = img_gen_t.flow_from_directory("../dataset/chest_xray/train-ready", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../dataset/chest_xray/test-ready", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

Found 4708 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/1
1177/1177 [==============================] - 337s 286ms/step - loss: 0.0348 - acc: 0.9870 - val_loss: 2.2327 - val_acc: 0.7196


### 5 load weights & predict

In [43]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [44]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

Found 14 images belonging to 3 classes.
0
1
[array([ 0.30176267,  0.35524741,  0.34298989], dtype=float32), array([ 0.28475785,  0.34978592,  0.36545616], dtype=float32), array([ 0.3018229 ,  0.35525236,  0.34292471], dtype=float32), array([ 0.29986629,  0.35433695,  0.34579682], dtype=float32)]


In [45]:
print(np.array(predictions))

[[ 0.30176267  0.35524741  0.34298989]
 [ 0.28475785  0.34978592  0.36545616]
 [ 0.3018229   0.35525236  0.34292471]
 [ 0.29986629  0.35433695  0.34579682]]


### 6 write predictions 2 CSV